0. Environment configuration

In [ ]:
import pandas as pd
from sqlalchemy import  create_engine

In [ ]:
df_dates = pd.read_csv('date_info.csv')

df_dates['calendar_date'] = pd.to_datetime(df_dates['calendar_date'],dayfirst=True)

df_rest_visitor = pd.read_csv('restaurants_visitors.csv')

df_rest_visitor = df_rest_visitor.loc[df_rest_visitor['visit_date'] != "#VALUE!"]

df_rest_visitor['visit_date'] = pd.to_datetime(df_rest_visitor['visit_date'],dayfirst=True)

df_store_info = pd.read_csv('store_info.csv')

In [ ]:
print (df_rest_visitor)

In [ ]:
print(df_dates)

In [ ]:
print(df_store_info)

In [ ]:
db_url = "sqlite:///:memory:"
engine = engine = create_engine(db_url)

df_dates.to_sql('dates', engine, index=False)
df_rest_visitor.to_sql('restaurant_visitors', engine, index=False)
df_store_info.to_sql('store_info',engine, index=False)

1. Write the SQL queries necessary to generate a list of the five restaurants that have the highest average number of visitors on holidays. The result table should also contain that average per restaurant

In [ ]:
query_1 = """
SELECT 
    a.[id] as restaurant,
    AVG(a.[reserve_visitors]) as average_of_visitors 
FROM restaurant_visitors as a 
LEFT JOIN dates AS b 
ON a. [visit_date] = b. [calendar_date] 
WHERE b.[holiday_flg]=1 
GROUP BY a.[id] 
ORDER BY average_of_visitors DESC 
LIMIT 5
"""

first_result = pd.read_sql_query(query_1,engine)

print (first_result)

2. Use SQL to discover which day of the week there are usually more visitors on average in restaurants.



In [ ]:
query_2 = """
SELECT 
    b.[day_of_week] ,
    AVG(a.[reserve_visitors]) as average_of_visitor 
FROM dates AS b 
LEFT JOIN restaurant_visitors AS a
ON b. [calendar_date] = a. [visit_date] 
GROUP BY b.[day_of_week] 
ORDER BY average_of_visitor DESC
"""

second_result = pd.read_sql_query(query_2,engine)

print (second_result)

3. How was the percentage of growth of the amount of visitors week over week for the last four weeks of the data? Use SQL too

In [ ]:
query_days = """
WITH amount_wow AS ( 
    SELECT
        DATE(b.calendar_date, '-' || strftime('%w', b.calendar_date) || ' days') AS week_start, 
        SUM(a.reserve_visitors) AS amount_visitors 
    FROM dates AS b
    LEFT JOIN restaurant_visitors AS a 
    ON b.calendar_date = a.visit_date
    GROUP BY week_start) 
SELECT  
    week_start,
    amount_visitors,
    LAG(amount_visitors) OVER (ORDER BY week_start) AS previous_week_amount,
    (amount_visitors * 1.00 -LAG(amount_visitors) OVER (ORDER BY week_start))/LAG(amount_visitors) OVER (ORDER BY week_start)  AS  growth_visitors_wow
FROM amount_wow 
ORDER BY week_start DESC 
LIMIT 4
"""

days_result = pd.read_sql_query(query_days,engine)

print (days_result)